## Population growth
* Populations that die are deleted, so they don't grow.
* However all pops that can grow will, regardless of wheter they are starving or not. 
* Starving will lower thier health, which will eventually make them unable to grow. 

Sarving and Death is taken care of elsewhere, so this notebook won't have that filter. 


In [1]:
import sys
import numpy as np
import pandas as pd

# mapping to the modules that make the app
sys.path.insert(0, "../..")



import yaml, ssl, asyncio, pickle, os, ast

ssl._create_default_https_context = ssl._create_unverified_context
asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())

import nest_asyncio
# this is required for running in a Jupyter Notebook. 
nest_asyncio.apply()

#importing the libraries from the app
from app.connectors import cmdb_graph
from app.objects import time
from app.objects import population, species
from app.functions import language, maths

c = cmdb_graph.CosmosdbClient()


executing local windows deployment


Much of the Population growth is handled in scripts inside the Azure Function

In [2]:
params = yaml.safe_load(open(os.path.join(os.getenv("abspath"),"app/configurations/popgrowthconfig.yaml")))
syllables = pickle.load(open(os.path.join(os.getenv("abspath"),"app/creators/specs/syllables.p"), "rb"))
username = 'notebook'

Time is updated by the `time` function. It is always running.

In [3]:
t = time.Time(c)
t.get_current_UTU()
params['currentTime'] = t.params['currentTime']
t

< time at: 2024-05-23T15:46:02.228975+00:00 UTU:15985 >

In [4]:
params

{'pop_health_requirement': 0.7,
 'pop_consumes': 2,
 'starve_damage': 0.05,
 'changing_values': ['conformity',
  'literacy',
  'aggression',
  'constitution',
  'wealth',
  'factionLoyalty'],
 'currentTime': 15985}

For this notebook, setting the `pop_health_requirement` to an arbitrary value just to examine.

In [27]:
params['pop_health_requirement'] = 0.4

In [36]:
healthy_pops_query = f"""
            g.V().has('label','pop')
                .has('health',gt({params.get("pop_health_requirement")})).as('pop')
                .values('objid','health','wealth')
            """
c.run_query(healthy_pops_query)

chop those results into a dataframe.

Note that if `c.res` is empty (because no pops match that criteria), you can't run this step. `ZeroDivisionError: float modulo`

In [37]:
a = np.array(c.res)
pops_df = pd.DataFrame(np.split(a,len(a)/3),columns=['objid','health','wealth'])
pops_df[['health','wealth']] = pops_df[['health','wealth']].astype(float)

pops_df

,objid,health,wealth
0,7092251861714,0.7,0.4798
1,5550515400694,0.7,0.4200
2,7880677119676,0.7,0.4810
3,5380932415285,0.7,0.4885
4,6684973836024,0.7,0.4960
5,7961679440441,0.7,0.4878
6,7082994639519,0.7,0.4578


Add randomness

In [38]:
pops_df['roll'] = pops_df['objid'].apply(lambda x: np.random.random())
pops_df

,objid,health,wealth,roll
0,7092251861714,0.7,0.4798,0.330520
1,5550515400694,0.7,0.4200,0.976218
2,7880677119676,0.7,0.4810,0.961471
3,5380932415285,0.7,0.4885,0.137446
4,6684973836024,0.7,0.4960,0.601826
5,7961679440441,0.7,0.4878,0.605896
6,7082994639519,0.7,0.4578,0.041130


select the populations that will grow. 

In [39]:
pops_df['grow'] = pops_df[['wealth','health']].T.mean() >= pops_df['roll']
pops_df

,objid,health,wealth,roll,grow
0,7092251861714,0.7,0.4798,0.330520,True
1,5550515400694,0.7,0.4200,0.976218,False
2,7880677119676,0.7,0.4810,0.961471,False
3,5380932415285,0.7,0.4885,0.137446,True
4,6684973836024,0.7,0.4960,0.601826,False
5,7961679440441,0.7,0.4878,0.605896,False
6,7082994639519,0.7,0.4578,0.041130,True


In [40]:
reproducing_pops = pops_df[pops_df['grow']].drop(['roll','grow'],axis=1).reset_index(drop=True)
reproducing_pops

,objid,health,wealth
0,7092251861714,0.7,0.4798
1,5380932415285,0.7,0.4885
2,7082994639519,0.7,0.4578


Now that we have the list of reproducing pops, we can generate new species from them. 


In [41]:
parent_pop = reproducing_pops.sample().to_dict(orient='records')[0]
parent_pop

{'objid': '7092251861714', 'health': 0.7, 'wealth': 0.4798}

We need to check that the location's population cap hasn't been reached. 

In [ ]:
We need to check that the pop 

Note, there are a specific list of edges that a child note must have. 

In [13]:
query_child_must_have = f"""
        g.V().has('objid','{parent_pop["objid"]}')
            .outE().values('label','inVLabel','outVLabel')
        """


c.run_query(query_child_must_have)
required_edges = c.res
required_edges

['inhabits', 'isOf', 'isIn', 'caused']

* 'isOf' - The Species
* 'inhabits' - planet or vessal that it is in
* 'childOf' - It's parent pop
* 'isIn' - The faction that it is in. 

In [14]:
query_pop_species_faction = f"""
        g.V().has('objid','{parent_pop["objid"]}')
            .local(
                union(
                    out('inhabits').as('location'),
                    out('isOf').as('species'),
                    out('isIn').as('faction')
                    )
                    .fold()).as('pop','location','species','faction')
                .path()
                .by(unfold().valueMap().fold())
        """


c.run_query(query_pop_species_faction)
c.res

[{'labels': [[], ['pop', 'location', 'species', 'faction']],
  'objects': [[{'objid': ['8471680871431'],
     'name': ['Jegongontes Yemay'],
     'conformity': [0.59],
     'literacy': [0.378],
     'aggression': [0.621],
     'constitution': [0.566],
     'health': [0.7],
     'isIn': ['None'],
     'industry': [0.5935],
     'wealth': [0.4858],
     'factionLoyalty': [0.48],
     'isIdle': ['true'],
     'userguid': ['ac5b8081-7ef9-4bce-baac-6d0ea7e1782c'],
     'objtype': ['pop']}],
   [{'name': ['Son'],
     'class': ['terrestrial'],
     'objid': ['6791071841421'],
     'atmosphere': ['[{Argon: 0.0}, {Carbon Dioxide: 0.457}, {Helium: 0.021}, {Hydrogen: 0.184}, {Methane: 0.004}, {Nitrogen: 0.169}, {Oxygen: 0.093}, {Sodium: 0.071}]'],
     'radius': [0.332],
     'mass': [0.467],
     'orbitsDistance': [1.041],
     'orbitsId': ['2290333943418'],
     'orbitsName': ['Elhaicu'],
     'isSupportsLife': ['true'],
     'isPopulated': ['true'],
     'isHomeworld': ['true'],
     'usergui

In [15]:
pop_dict = c.clean_node(c.res[0]['objects'][0][0])
pop_dict

{'objid': '8471680871431',
 'name': 'Jegongontes Yemay',
 'conformity': 0.59,
 'literacy': 0.378,
 'aggression': 0.621,
 'constitution': 0.566,
 'health': 0.7,
 'isIn': 'None',
 'industry': 0.5935,
 'wealth': 0.4858,
 'factionLoyalty': 0.48,
 'isIdle': 'true',
 'userguid': 'ac5b8081-7ef9-4bce-baac-6d0ea7e1782c',
 'objtype': 'pop',
 'id': '8471680871431'}

In [16]:
loc_dict = c.clean_node(c.res[0]['objects'][1][0])
loc_dict

{'name': 'Son',
 'class': 'terrestrial',
 'objid': '6791071841421',
 'atmosphere': '[{Argon: 0.0}, {Carbon Dioxide: 0.457}, {Helium: 0.021}, {Hydrogen: 0.184}, {Methane: 0.004}, {Nitrogen: 0.169}, {Oxygen: 0.093}, {Sodium: 0.071}]',
 'radius': 0.332,
 'mass': 0.467,
 'orbitsDistance': 1.041,
 'orbitsId': '2290333943418',
 'orbitsName': 'Elhaicu',
 'isSupportsLife': 'true',
 'isPopulated': 'true',
 'isHomeworld': 'true',
 'userguid': 'ac5b8081-7ef9-4bce-baac-6d0ea7e1782c',
 'objtype': 'planet',
 'id': '6791071841421'}

In [17]:
sp_dict = c.clean_node(c.res[0]['objects'][1][1])
sp_dict

{'name': 'Viaren',
 'objid': '5373646850758',
 'consumes': 'organics',
 'effuses': 'organic waste,plastics',
 'conformity': '0.290',
 'aggression': '0.750',
 'literacy': '0.250',
 'constitution': '0.710',
 'userguid': 'ac5b8081-7ef9-4bce-baac-6d0ea7e1782c',
 'objtype': 'species',
 'id': '5373646850758'}

In [18]:
fact_dict = c.clean_node(c.res[0]['objects'][1][2])
fact_dict

{'name': 'Jegongontes',
 'objid': '4107450191617',
 'lat': 0.054,
 'long': 0.254,
 'userguid': 'ac5b8081-7ef9-4bce-baac-6d0ea7e1782c',
 'objtype': 'faction',
 'id': '4107450191617'}

Note that the formatting (created by the structure of the query is)
```
[
    [pop],
    [species,faction]
]
```

Going to create species objects for the populations that will reproduce

In [19]:
sp = species.Species(sp_dict)
sp

<species: None; 5373646850758; Viaren>

Loading in the parent attributes as defaults

In [20]:
sp.config['defaults'] = pop_dict

In [21]:
sp.config

{'name': 'Viaren',
 'objid': '5373646850758',
 'consumes': 'organics',
 'effuses': 'organic waste,plastics',
 'conformity': '0.290',
 'aggression': '0.750',
 'literacy': '0.250',
 'constitution': '0.710',
 'userguid': 'ac5b8081-7ef9-4bce-baac-6d0ea7e1782c',
 'objtype': 'species',
 'id': '5373646850758',
 'defaults': {'objid': '8471680871431',
  'name': 'Jegongontes Yemay',
  'conformity': 0.59,
  'literacy': 0.378,
  'aggression': 0.621,
  'constitution': 0.566,
  'health': 0.7,
  'isIn': 'None',
  'industry': 0.5935,
  'wealth': 0.4858,
  'factionLoyalty': 0.48,
  'isIdle': 'true',
  'userguid': 'ac5b8081-7ef9-4bce-baac-6d0ea7e1782c',
  'objtype': 'pop',
  'id': '8471680871431'}}

In [22]:
child = population.Pop(sp)
child.name = sp.config['defaults']['name']+language.make_word(1).lower()
child

<pop: pop; 9956377096406; Jegongontes Yemaynam>

Note that the child must have a userguid

In [23]:
child.get_data()

{'name': 'Jegongontes Yemaynam',
 'objid': '9956377096406',
 'label': 'pop',
 'conformity': 0.44,
 'literacy': 0.46,
 'aggression': 0.437,
 'constitution': 0.524,
 'health': 0.7,
 'isIn': None,
 'industry': 0.48050000000000004,
 'wealth': 0.47025000000000006,
 'factionLoyalty': 0.347,
 'isIdle': True}

Now that I have the child population, I can check that they have the needed edges. 

In [24]:
required_edges

['inhabits', 'isOf', 'isIn', 'caused']

In [25]:
child.childOf

{'node1': '9956377096406', 'node2': '8471680871431', 'label': 'childOf'}

In [26]:
child.isOfSpecies

{'node1': '9956377096406', 'node2': '5373646850758', 'label': 'isOf'}

In [27]:
isIn_edge = child.get_isInFaction()
isIn_edge['node2'] = fact_dict['objid']
isIn_edge

{'node1': '9956377096406', 'node2': '4107450191617', 'label': 'isIn'}

In [28]:
inhabits_edge = {'node1': child.objid, 'node2': loc_dict['objid'], 'label': 'inhabits'}
inhabits_edge

{'node1': '9956377096406', 'node2': '6791071841421', 'label': 'inhabits'}

In [29]:
pd.DataFrame([child.childOf,
              child.isOfSpecies,
              isIn_edge,
              inhabits_edge])

,node1,node2,label
0,9956377096406,8471680871431,childOf
1,9956377096406,5373646850758,isOf
2,9956377096406,4107450191617,isIn
3,9956377096406,6791071841421,inhabits


## Building the uploaded data

In [30]:
child_data = child.get_data()
child_data['userguid'] = pop_dict['userguid'] 
child_data

{'name': 'Jegongontes Yemaynam',
 'objid': '9956377096406',
 'label': 'pop',
 'conformity': 0.44,
 'literacy': 0.46,
 'aggression': 0.437,
 'constitution': 0.524,
 'health': 0.7,
 'isIn': None,
 'industry': 0.48050000000000004,
 'wealth': 0.47025000000000006,
 'factionLoyalty': 0.347,
 'isIdle': True,
 'userguid': 'ac5b8081-7ef9-4bce-baac-6d0ea7e1782c'}

In [31]:
def population_growth_event(t,parent,location,child):
    node = {
        'objid':maths.uuid(),
        'name':'population growth',
        'label':'event',
        'text': f"The population ({parent['name']}) inhabiting {location['name']} has grown to produce the population: {child.name}.",
        'visibleTo':parent['userguid'],
        'time':t.params['currentTime'],
        'userguid':parent['userguid'],
        'source':'notebook'
    }
    return node

event = population_growth_event(t,pop_dict,loc_dict,child)
event

{'objid': '4417806903516',
 'name': 'population growth',
 'label': 'event',
 'text': 'The population (Jegongontes Yemay) inhabiting Son has grown to produce the population: Jegongontes Yemaynam.',
 'visibleTo': 'ac5b8081-7ef9-4bce-baac-6d0ea7e1782c',
 'time': 13989,
 'userguid': 'ac5b8081-7ef9-4bce-baac-6d0ea7e1782c',
 'source': 'notebook'}

In [32]:
event_edge = {'node1': child.objid, 'node2': event['objid'], 'label': 'caused'}

In [33]:
data = {"nodes":[child_data, event] ,"edges":[inhabits_edge,child.isOfSpecies,isIn_edge,event_edge] }

print(f"The final dataset is {len(data.get('nodes'))} nodes and {len(data.get('edges'))} edges")

The final dataset is 2 nodes and 4 edges


In [34]:
pd.DataFrame(data['edges'])

,node1,node2,label
0,9956377096406,6791071841421,inhabits
1,9956377096406,5373646850758,isOf
2,9956377096406,4107450191617,isIn
3,9956377096406,4417806903516,caused


In [35]:
pd.DataFrame(data['nodes']) 

,name,objid,label,conformity,literacy,aggression,constitution,health,isIn,industry,wealth,factionLoyalty,isIdle,userguid,text,visibleTo,time,source
0,Jegongontes Yemaynam,9956377096406,pop,0.44,0.46,0.437,0.524,0.7,NaN,0.4805,0.47025,0.347,True,ac5b8081-7ef9-4bce-baac-6d0ea7e1782c,NaN,NaN,NaN,NaN
1,population growth,4417806903516,event,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ac5b8081-7ef9-4bce-baac-6d0ea7e1782c,The population (Jegongontes Yemay) inhabiting ...,ac5b8081-7ef9-4bce-baac-6d0ea7e1782c,13989.0,notebook


It takes a while to upload all of that data. 

In [36]:
# c.upload_data('notebook',data)

# Testing the actual function

In [37]:
from app.functions import growth

In [43]:
parents = growth.calculate_growth(c,t,params)
parents

[{'agent': {'objid': '0804013652458', 'health': 0.7, 'wealth': 0.4728},
  'action': 'reproduce'},
 {'agent': {'objid': '5840971700979', 'health': 0.7, 'wealth': 0.5397},
  'action': 'reproduce'},
 {'agent': {'objid': '2797495206916', 'health': 0.7, 'wealth': 0.5662},
  'action': 'reproduce'},
 {'agent': {'objid': '5586452954582', 'health': 0.7, 'wealth': 0.5392},
  'action': 'reproduce'},
 {'agent': {'objid': '9329901454713', 'health': 0.7, 'wealth': 0.4938},
  'action': 'reproduce'},
 {'agent': {'objid': '9561682264646', 'health': 0.7, 'wealth': 0.6407},
  'action': 'reproduce'},
 {'agent': {'objid': '6031502657352', 'health': 0.7, 'wealth': 0.2995},
  'action': 'reproduce'},
 {'agent': {'objid': '4809768309289', 'health': 0.7, 'wealth': 0.4912},
  'action': 'reproduce'},
 {'agent': {'objid': '6429552640412', 'health': 0.7, 'wealth': 0.431},
  'action': 'reproduce'},
 {'agent': {'objid': '6965718418473', 'health': 0.7, 'wealth': 0.6225},
  'action': 'reproduce'},
 {'agent': {'objid': 

In [44]:
[p['agent'] for p in parents]

[{'objid': '0804013652458', 'health': 0.7, 'wealth': 0.4728},
 {'objid': '5840971700979', 'health': 0.7, 'wealth': 0.5397},
 {'objid': '2797495206916', 'health': 0.7, 'wealth': 0.5662},
 {'objid': '5586452954582', 'health': 0.7, 'wealth': 0.5392},
 {'objid': '9329901454713', 'health': 0.7, 'wealth': 0.4938},
 {'objid': '9561682264646', 'health': 0.7, 'wealth': 0.6407},
 {'objid': '6031502657352', 'health': 0.7, 'wealth': 0.2995},
 {'objid': '4809768309289', 'health': 0.7, 'wealth': 0.4912},
 {'objid': '6429552640412', 'health': 0.7, 'wealth': 0.431},
 {'objid': '6965718418473', 'health': 0.7, 'wealth': 0.6225},
 {'objid': '1173164904813', 'health': 0.7, 'wealth': 0.5337},
 {'objid': '2144284794318', 'health': 0.7, 'wealth': 0.513},
 {'objid': '0668489712341', 'health': 0.7, 'wealth': 0.5625},
 {'objid': '8471680871431', 'health': 0.7, 'wealth': 0.4858},
 {'objid': '8291246620344', 'health': 0.7, 'wealth': 0.367},
 {'objid': '5707178019607', 'health': 0.7, 'wealth': 0.4838},
 {'objid': 

In [46]:
dfs = []
for p in [p['agent'] for p in parents]:
    data = growth.grow_population(c,t, p)
    dfs.append(data)

In [47]:
data

{'nodes': [{'name': 'Jegongontes Yemayryhia',
   'objid': '9112996930579',
   'label': 'pop',
   'conformity': 0.501,
   'literacy': 0.631,
   'aggression': 0.403,
   'constitution': 0.425,
   'health': 0.7,
   'isIn': None,
   'industry': 0.41400000000000003,
   'wealth': 0.5225,
   'factionLoyalty': 0.637,
   'isIdle': 'true',
   'userguid': 'ac5b8081-7ef9-4bce-baac-6d0ea7e1782c',
   'id': '9112996930579'},
  {'objid': '2908960875305',
   'name': 'population growth',
   'label': 'event',
   'text': 'The population (Jegongontes Yemayry) inhabiting Son has grown to produce the population: Jegongontes Yemayryhia.',
   'visibleTo': 'ac5b8081-7ef9-4bce-baac-6d0ea7e1782c',
   'time': 13989,
   'userguid': 'ac5b8081-7ef9-4bce-baac-6d0ea7e1782c',
   'source': 'notebook',
   'id': '2908960875305'}],
 'edges': [{'node1': '9112996930579',
   'node2': '6791071841421',
   'label': 'inhabits'},
  {'node1': '9112996930579', 'node2': '5373646850758', 'label': 'isOf'},
  {'node1': '9112996930579', 'n

In [48]:
pd.DataFrame([d['nodes'] for d in dfs])

,0,1
0,"{'name': 'Tadttartrosia Balwalpin', 'objid': '...","{'objid': '7048518277044', 'name': 'population..."
1,"{'name': 'Tadttartrosia Bridlis', 'objid': '58...","{'objid': '3512691701424', 'name': 'population..."
2,"{'name': 'Tadttartrosia Koygerva', 'objid': '7...","{'objid': '8621950387977', 'name': 'population..."
3,"{'name': 'Jegongontes Yequi', 'objid': '337637...","{'objid': '3534719503579', 'name': 'population..."
4,"{'name': 'Jegongontes Vilbo', 'objid': '765259...","{'objid': '3185478274113', 'name': 'population..."
5,"{'name': 'Peebat Koysamganmesay', 'objid': '40...","{'objid': '6579067285507', 'name': 'population..."
6,"{'name': 'Mosfo Tuhes', 'objid': '468314559353...","{'objid': '3113860578798', 'name': 'population..."
7,"{'name': 'Jegongontes Jebil', 'objid': '755578...","{'objid': '4472386620556', 'name': 'population..."
8,"{'name': 'Jegongontes Yesernam', 'objid': '459...","{'objid': '6848352203739', 'name': 'population..."
9,"{'name': 'Jegongontes Yereine', 'objid': '8374...","{'objid': '3512331476769', 'name': 'population..."
